# Problem 1
Problem 1 - SSD, ONNX model, Visualization, Inferencing - 25 points

In this problem we will be inferencing SSD ONNX model using ONNX Runtime Server. You will follow the github repo and ONNX tutorials (links provided below). You will start with a pretrained Pytorch SSD model and retrain it for your target categories. Then you will convert this Pytorch model to ONNX and deploy it on ONNX runtime server for inferencing.

## 1
Download pretrained pytorch MobilenetV1 SSD and test it locally using Pascal VOC 2007 dataset.

Show the test accuracy for the 20 classes. (3)

In [2]:
import torchvision

In [3]:
test_dataset = torchvision.datasets.VOCDetection(root="./cached_datasets/", year="2007", image_set="test", download=True)

100%|██████████| 451020800/451020800 [00:23<00:00, 19024674.39it/s]


Extracting ./cached_datasets/VOCtest_06-Nov-2007.tar to ./cached_datasets


## 2
Select any two related categories from Google Open Images dataset and finetune the pretrained SSD model. Examples include, Aircraft and Aeroplane, Handgun and Shotgun. You can use open_images_downloader.py script provided at the github to download the data. For finetuning you can use the same parameters as in the tutorial below. Compute the accuracy of the test data for these categories before and after finetuning. (4+4)

In [2]:
import torch

### Before Finetuning

<!-- Before Finetuning, the final classification layer dim is different, we can not directly use the pretrained model to evaluate. We use the epoch=0 (fitunied 1 epoch) model to evaluate. -->

### After Finetuning

## 3
Convert the Pytorch model to ONNX format and save it. (2)

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
import torch.onnx as torch_onnx

# class Model(nn.Module):
#     def __init__(self):
#         super(Model, self).__init__()
#         self.conv = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=(3,3), stride=1, padding=0, bias=False)

#     def forward(self, inputs):
#         x = self.conv(inputs)
#         #x = x.view(x.size()[0], x.size()[1], -1)
#         return torch.mean(x, dim=2)

# # Use this an input trace to serialize the model
# input_shape = (3, 100, 100)
# model_onnx_path = "torch_model.onnx"
# model = Model()
# model.train(False)

# # Export the model to an ONNX file
# dummy_input = Variable(torch.randn(1, *input_shape))
# output = torch_onnx.export(model, 
#                           dummy_input, 
#                           model_onnx_path, 
#                           verbose=False)
# print("Export of torch_model.onnx complete!")

In [14]:
cd "third_party/pytorch-ssd"

/home/xiangpan/Labs/NYU_DL_Sys/HW5/third_party/pytorch-ssd


In [15]:
pwd

'/home/xiangpan/Labs/NYU_DL_Sys/HW5/third_party/pytorch-ssd'

In [16]:
from vision.ssd.mobilenetv1_ssd import create_mobilenetv1_ssd

In [19]:
model = create_mobilenetv1_ssd(num_classes=21)

In [20]:
state_dict_path = "/home/xiangpan/Labs/NYU_DL_Sys/HW5/cached_models/mobilenet-v1-ssd-mp-0_675.pth"
state_dict = torch.load(state_dict_path)

In [21]:
model.load_state_dict(state_dict)

<All keys matched successfully>

In [22]:
model

SSD(
  (base_net): Sequential(
    (0): Sequential(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
    )
    (1): Sequential(
      (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(32, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
    )
    (2): Sequential(
      (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=64, bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(64, 128, kernel_size=(1, 1), st

In [24]:
model.train(False)

model_onnx_path = "/home/xiangpan/Labs/NYU_DL_Sys/HW5/cached_models/mobilenet-v1-ssd-mp-0_675.onnx"
# Export the model to an ONNX file
input_shape = (3, 300, 300)
dummy_input = Variable(torch.randn(1, *input_shape))
output = torch_onnx.export(model, 
                          dummy_input, 
                          model_onnx_path, 
                          verbose=False)
print("Export of torch_model.onnx complete!")

Export of torch_model.onnx complete!


## 4
Visualize the model using net drawer tool. Compile the model using embed_docstring flag and show the visualization output. Also show doc string (stack trace for PyTorch) for different types of nodes. (4)

### visualize_model

#### Without embed_docstring

<img src="./problem1/mb_v1_ssd_without_flag.png">

#### With embed_docstring

<img src="./problem1/mb_v1_ssd_with_flag.png">

### compile
Compile the model using embed_docstring flag and show the visualization output. Also show doc string (stack trace for PyTorch) for different types of nodes.

TODO!!!

#### doc string 

In [35]:
cd /home/xiangpan/Labs/NYU_DL_Sys/HW5

/home/xiangpan/Labs/NYU_DL_Sys/HW5


In [36]:
torch_model = model

from torch.autograd import Variable
batch_size = 1    # just a random number

# Input to the model
x = Variable(torch.randn(batch_size, 3, 224, 224), requires_grad=True)

# Export the model
torch_out = torch.onnx._export(torch_model,             # model being run
                               x,                       # model input (or a tuple for multiple inputs)
                               "./cached_models/mobilenet-v1-ssd-mp-0_675.onnx",       # where to save the model (can be a file or file-like object)
                               export_params=True)      # store the trained parameter weights inside the model file

## 5
Deploy the ONNX model on ONNX runtime (ORT) server. You need to set up the environment following steps listed in the tutorial. Then you need make HTTP request to the ORT server. Test the inferencing set-up using 1 image from each of the two selected categories. (4)

systemctl start docker

sudo docker pull mcr.microsoft.com/onnxruntime/server



## 6
Parse the response message from the ORT server and annotate the two images. Show inferencing output (bounding boxes with labels) for the two images. (4)